# US Used Cars (2006 - 2020)

This dataset contains detailed information on approximately 3 million used car listings across the United States from 2005–2020. It is provided as a well-structured CSV (about 9 GB) with dozens of features per car

The dataset provides ~66 features per car, encompassing numeric attributes (mileage, engine size, year, horsepower, etc.) and categorical descriptors (make, model, body type, fuel type, etc.). This richness enables creative feature engineering and the exploration of multiple feature types (numeric, categorical,
ordinal).

## Data Preprocessing

The data is already in a cleaned, structured format (compiled via a web crawler from CarGurus listings). Most fields are well-defined and ready to use. Only a few columns have missing values (e.g. ~9 features with >50% missing can be dropped upfront). And standard techniques like imputation can address any
remaining gaps. No extensive data scraping or text parsing is needed to get started.

### Loading

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import ast
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from catboost import CatBoostRegressor, Pool
from sklearn.impute import SimpleImputer
from category_encoders import TargetEncoder
import warnings
warnings.filterwarnings("ignore")

In [2]:
import kagglehub

In [3]:
path = kagglehub.dataset_download("ananaymital/us-used-cars-dataset")
print("Path to dataset files:", path)

Path to dataset files: C:\Users\adand\.cache\kagglehub\datasets\ananaymital\us-used-cars-dataset\versions\1


In [4]:
df_path = r"C:\Users\adand\.cache\kagglehub\datasets\ananaymital\us-used-cars-dataset\versions\1\used_cars_data.csv"

In [5]:
cars_df = pd.read_csv(df_path)

In [6]:
cars_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000040 entries, 0 to 3000039
Data columns (total 66 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   vin                      object 
 1   back_legroom             object 
 2   bed                      object 
 3   bed_height               object 
 4   bed_length               object 
 5   body_type                object 
 6   cabin                    object 
 7   city                     object 
 8   city_fuel_economy        float64
 9   combine_fuel_economy     float64
 10  daysonmarket             int64  
 11  dealer_zip               object 
 12  description              object 
 13  engine_cylinders         object 
 14  engine_displacement      float64
 15  engine_type              object 
 16  exterior_color           object 
 17  fleet                    object 
 18  frame_damaged            object 
 19  franchise_dealer         bool   
 20  franchise_make           object 
 21  front_le

In [7]:
cars_df.describe()

,city_fuel_economy,combine_fuel_economy,daysonmarket,engine_displacement,highway_fuel_economy,horsepower,is_certified,latitude,listing_id,longitude,mileage,owner_count,price,savings_amount,seller_rating,sp_id,vehicle_damage_category,year
count,2.508755e+06,0.0,3.000040e+06,2.827654e+06,2.508755e+06,2.827654e+06,0.0,3.000040e+06,3.000040e+06,3.000040e+06,2.855653e+06,1.483027e+06,3.000040e+06,3.000040e+06,2.959168e+06,2.999944e+06,0.0,3.000040e+06
mean,2.269325e+01,NaN,7.605973e+01,2.968488e+03,2.947337e+01,2.479957e+02,NaN,3.698414e+01,2.754987e+08,-9.064224e+01,3.114690e+04,1.532644e+00,2.993337e+04,5.509768e+02,4.270413e+00,2.335224e+05,NaN,2.017728e+03
std,8.807024e+00,NaN,1.088839e+02,1.348905e+03,7.769252e+00,9.046639e+01,NaN,4.996819e+00,8.894123e+06,1.390589e+01,7.458675e+04,9.202928e-01,1.956617e+04,1.079448e+03,5.133017e-01,1.323221e+05,NaN,4.178701e+00
min,7.000000e+00,NaN,0.000000e+00,7.000000e+02,1.000000e+01,5.500000e+01,NaN,1.834670e+01,1.994620e+07,-1.579280e+02,0.000000e+00,1.000000e+00,1.650000e+02,0.000000e+00,1.000000e+00,4.159300e+04,NaN,1.915000e+03
25%,1.800000e+01,NaN,1.400000e+01,2.000000e+03,2.500000e+01,1.750000e+02,NaN,3.350920e+01,2.745794e+08,-9.708820e+01,6.000000e+00,1.000000e+00,1.845100e+04,0.000000e+00,4.000000e+00,6.337500e+04,NaN,2.017000e+03
50%,2.100000e+01,NaN,3.500000e+01,2.500000e+03,2.900000e+01,2.440000e+02,NaN,3.784710e+01,2.785453e+08,-8.724950e+01,8.267000e+03,1.000000e+00,2.647700e+04,0.000000e+00,4.341463e+00,2.816270e+05,NaN,2.020000e+03
75%,2.600000e+01,NaN,8.200000e+01,3.600000e+03,3.300000e+01,3.000000e+02,NaN,4.100620e+01,2.804553e+08,-8.045490e+01,4.366200e+04,2.000000e+00,3.822000e+04,7.850000e+02,4.605263e+00,3.366140e+05,NaN,2.020000e+03
max,1.270000e+02,NaN,3.599000e+03,8.400000e+03,1.270000e+02,1.001000e+03,NaN,6.120310e+01,2.820222e+08,-6.607850e+01,9.999999e+07,1.900000e+01,3.299995e+06,1.474140e+05,5.000000e+00,4.409510e+05,NaN,2.021000e+03


In [8]:
cars_df.head(10)

,vin,back_legroom,bed,bed_height,bed_length,body_type,cabin,city,city_fuel_economy,combine_fuel_economy,...,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,ZACNJABB5KPJ92081,35.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,Bayamon,NaN,NaN,...,A,9-Speed Automatic Overdrive,t83804,Latitude FWD,NaN,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019
1,SALCJ2FX1LH858117,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
2,JF1VA2M67G9829723,35.4 in,NaN,NaN,NaN,Sedan,NaN,Guaynabo,17.0,NaN,...,M,6-Speed Manual,t58994,Base,NaN,AWD,All-Wheel Drive,104.3 in,78.9 in,2016
3,SALRR2RV0L2433391,37.6 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,8-Speed Automatic Overdrive,t86074,V6 HSE AWD,NaN,AWD,All-Wheel Drive,115 in,87.4 in,2020
4,SALCJ2FXXLH862327,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
5,SALYK2EX1LA261711,37.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,8-Speed Automatic Overdrive,t85614,P250 R-Dynamic S AWD,NaN,AWD,All-Wheel Drive,113.1 in,84.4 in,2020
6,3MZBPABL6KM107908,35.1 in,NaN,NaN,NaN,Sedan,NaN,Bayamon,NaN,NaN,...,A,6-Speed Automatic Overdrive,t85256,Sedan FWD,NaN,FWD,Front-Wheel Drive,107.3 in,70.7 in,2019
7,SALYK2EX5LA275434,37.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,8-Speed Automatic Overdrive,t85614,P250 R-Dynamic S AWD,NaN,AWD,All-Wheel Drive,113.1 in,84.4 in,2020
8,SALCJ2FX6LH858128,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
9,SALZL2GX4LH007593,33.8 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,9-Speed Automatic Overdrive,t85531,P300 R-Dynamic SE AWD,NaN,AWD,All-Wheel Drive,105.6 in,82.7 in,2020


In [9]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### Cleaning

In [10]:
missing_percentages = (cars_df.isnull().sum() / len(cars_df)) * 100
sorted_missing_percentages = missing_percentages.sort_values(ascending=False)
print(sorted_missing_percentages)

is_certified               100.000000
combine_fuel_economy       100.000000
vehicle_damage_category    100.000000
bed                         99.347742
cabin                       97.882262
is_oemcpo                   95.487993
is_cpo                      93.903481
bed_length                  85.696924
bed_height                  85.696924
owner_count                 50.566426
salvage                     47.552533
theft_title                 47.552533
frame_damaged               47.552533
fleet                       47.552533
has_accidents               47.552533
isCab                       47.552533
franchise_make              19.087579
torque                      17.259537
city_fuel_economy           16.375948
highway_fuel_economy        16.375948
power                       16.047319
interior_color              12.799363
main_picture_url            12.302936
major_options                6.668178
engine_displacement          5.746123
horsepower                   5.746123
width       

Dropping these columns as their missing values are extremely high

In [11]:
missing_threshold = 1
missing_percentages = cars_df.isnull().sum() / len(cars_df)
columns_to_drop = missing_percentages[missing_percentages >= missing_threshold].index
cars_df.drop(columns=columns_to_drop, inplace=True)
print(f"Columns dropped: {list(columns_to_drop)}")
print(f"New shape of cars_df: {cars_df.shape}")

Columns dropped: ['combine_fuel_economy', 'is_certified', 'vehicle_damage_category']
New shape of cars_df: (3000040, 63)


We lost 3 columns

In [12]:
cars_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000040 entries, 0 to 3000039
Data columns (total 63 columns):
 #   Column                Dtype  
---  ------                -----  
 0   vin                   object 
 1   back_legroom          object 
 2   bed                   object 
 3   bed_height            object 
 4   bed_length            object 
 5   body_type             object 
 6   cabin                 object 
 7   city                  object 
 8   city_fuel_economy     float64
 9   daysonmarket          int64  
 10  dealer_zip            object 
 11  description           object 
 12  engine_cylinders      object 
 13  engine_displacement   float64
 14  engine_type           object 
 15  exterior_color        object 
 16  fleet                 object 
 17  frame_damaged         object 
 18  franchise_dealer      bool   
 19  franchise_make        object 
 20  front_legroom         object 
 21  fuel_tank_volume      object 
 22  fuel_type             object 
 23  has_acc

### Cleaning II

In [13]:
# Check for duplicates using VIN column
print(f"Total rows before duplicate removal: {len(cars_df)}")
print(f"Duplicate rows based on VIN: {cars_df.duplicated(subset=['vin']).sum()}")

# Drop duplicates based on VIN column
cars_df = cars_df.drop_duplicates(subset=['vin'], keep='first')
print(f"Total rows after duplicate removal: {len(cars_df)}")

Total rows before duplicate removal: 3000040
Duplicate rows based on VIN: 40
Total rows after duplicate removal: 3000000


In [14]:
columns_to_drop = ['vin', 'description', 'latitude', 'longitude', 'listing_id', 
                   'main_picture_url', 'power', 'savings_amount', 'sp_id', 
                   'trimId', 'wheel_system_display', 'engine_type', 'exterior_color']

cars_df = cars_df.drop(columns=columns_to_drop)
print(f"Columns dropped: {columns_to_drop}")
print(f"New shape of cars_df: {cars_df.shape}")

Columns dropped: ['vin', 'description', 'latitude', 'longitude', 'listing_id', 'main_picture_url', 'power', 'savings_amount', 'sp_id', 'trimId', 'wheel_system_display', 'engine_type', 'exterior_color']
New shape of cars_df: (3000000, 50)


In [15]:
# Remove "in" from dimension columns
dimension_cols = ['back_legroom', 'front_legroom', 'height', 'length', 'wheelbase', 'width']
for col in dimension_cols:
    if col in cars_df.columns:
        cars_df[col] = cars_df[col].astype(str).str.replace(' in', '', regex=False)

# Remove "gal" from fuel_tank_volume
if 'fuel_tank_volume' in cars_df.columns:
    cars_df['fuel_tank_volume'] = cars_df['fuel_tank_volume'].astype(str).str.replace(' gal', '', regex=False)

# Remove "--" from all columns and convert to numeric
all_cols = dimension_cols + ['fuel_tank_volume']
for col in all_cols:
    if col in cars_df.columns:
        cars_df[col] = cars_df[col].astype(str).str.replace('--', '', regex=False)
        cars_df[col] = pd.to_numeric(cars_df[col], errors='coerce')

In [16]:
# Check for remaining "--" values in all columns
dash_count = 0
for col in cars_df.columns:
    if cars_df[col].dtype == 'object':  # Only check string/object columns
        dash_values = cars_df[col].astype(str).str.contains('--', na=False).sum()
        if dash_values > 0:
            print(f"Column '{col}': {dash_values} rows contain '--'")
            dash_count += dash_values

if dash_count == 0:
    print("No '--' values found in any column")
else:
    print(f"Total '--' values found: {dash_count}")

Column 'bed_height': 429091 rows contain '--'
Column 'bed_length': 8917 rows contain '--'
Column 'interior_color': 35 rows contain '--'
Column 'maximum_seating': 501 rows contain '--'
Total '--' values found: 438544


In [17]:
# Clean remaining "--" values from all columns
for col in cars_df.columns:
    if cars_df[col].dtype == 'object':
        cars_df[col] = cars_df[col].astype(str).str.replace('--', '', regex=False)
        # Convert to numeric if possible, otherwise keep as string
        cars_df[col] = pd.to_numeric(cars_df[col], errors='ignore')

# Verify cleanup
dash_count = 0
for col in cars_df.columns:
    if cars_df[col].dtype == 'object':
        dash_values = cars_df[col].astype(str).str.contains('--', na=False).sum()
        if dash_values > 0:
            print(f"Column '{col}': {dash_values} rows contain '--'")
            dash_count += dash_values

if dash_count == 0:
    print("All '--' values successfully removed")

All '--' values successfully removed


In [18]:
cars_df['maximum_seating'].value_counts()

maximum_seating
5 seats     1875332
7 seats      305325
6 seats      301481
8 seats      200358
nan          159266
4 seats       81173
2 seats       35105
3 seats       29676
15 seats       5906
9 seats        3634
12 seats       1703
10 seats        540
                501
Name: count, dtype: int64

In [19]:
# Remove "seats" from maximum_seating column
cars_df['maximum_seating'] = cars_df['maximum_seating'].astype(str).str.replace(' seats', '', regex=False)

In [20]:
# Comprehensive data type conversions

# Convert specified columns to float16
float16_cols = ['bed_height', 'bed_length', 'back_legroom', 'city_fuel_economy', 'highway_fuel_economy', 
                'front_legroom', 'fuel_tank_volume', 'height', 'wheelbase', 'width', 'length', 'seller_rating']

for col in float16_cols:
    if col in cars_df.columns:
        cars_df[col] = pd.to_numeric(cars_df[col], errors='coerce').astype('float16')

# Convert potential boolean columns
bool_cols = ['bed', 'cabin', 'fleet', 'frame_damaged', 'has_accidents', 'isCab', 
             'is_cpo', 'is_oemcpo', 'salvage', 'theft_title']

for col in bool_cols:
    if col in cars_df.columns:
        cars_df[col] = cars_df[col].map({'True': True, 'False': False, 'true': True, 'false': False,
                                        'Yes': True, 'No': False, 'yes': True, 'no': False,
                                        '1': True, '0': False, 1: True, 0: False})
        cars_df[col] = cars_df[col].astype('boolean')

# Convert integer columns
cars_df['year'] = cars_df['year'].astype('int16')
cars_df['daysonmarket'] = cars_df['daysonmarket'].astype('int16')
cars_df['horsepower'] = pd.to_numeric(cars_df['horsepower'], errors='coerce').astype('Int16')
cars_df['engine_displacement'] = pd.to_numeric(cars_df['engine_displacement'], errors='coerce').astype('Int16')
cars_df['maximum_seating'] = pd.to_numeric(cars_df['maximum_seating'], errors='coerce').astype('Int16')
cars_df['owner_count'] = pd.to_numeric(cars_df['owner_count'], errors='coerce').astype('Int8')
cars_df['mileage'] = pd.to_numeric(cars_df['mileage'], errors='coerce').astype('Int32')
cars_df['price'] = pd.to_numeric(cars_df['price'], errors='coerce').round().astype('Int32')

# Convert categorical columns
categorical_cols = ['body_type', 'fuel_type', 'transmission', 'transmission_display', 
                   'wheel_system', 'listing_color', 'interior_color']

for col in categorical_cols:
    if col in cars_df.columns:
        cars_df[col] = cars_df[col].astype('category')

# Convert datetime column
cars_df['listed_date'] = pd.to_datetime(cars_df['listed_date'], errors='coerce')

print("All data type conversions completed")

All data type conversions completed


In [21]:
cars_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3000000 entries, 0 to 3000039
Data columns (total 50 columns):
 #   Column                Dtype         
---  ------                -----         
 0   back_legroom          float16       
 1   bed                   boolean       
 2   bed_height            float16       
 3   bed_length            float16       
 4   body_type             category      
 5   cabin                 boolean       
 6   city                  object        
 7   city_fuel_economy     float16       
 8   daysonmarket          int16         
 9   dealer_zip            object        
 10  engine_cylinders      object        
 11  engine_displacement   Int16         
 12  fleet                 boolean       
 13  frame_damaged         boolean       
 14  franchise_dealer      bool          
 15  franchise_make        object        
 16  front_legroom         float16       
 17  fuel_tank_volume      float16       
 18  fuel_type             category      
 19  has_a

In [22]:
# Drop bed, bed_height, bed_length
columns_to_drop = ['bed', 'bed_height', 'bed_length', 'cabin', 'is_cpo', 'is_oemcpo', 'interior_color']
cars_df = cars_df.drop(columns=columns_to_drop)
print(f"Columns dropped: {columns_to_drop}")
print(f"New shape of cars_df: {cars_df.shape}")

Columns dropped: ['bed', 'bed_height', 'bed_length', 'cabin', 'is_cpo', 'is_oemcpo', 'interior_color']
New shape of cars_df: (3000000, 43)


In [23]:
# Print number of unique values for each column
for col in cars_df.columns:
    unique_count = cars_df[col].nunique()
    print(f"{col}: {unique_count} unique values")

back_legroom: 218 unique values
body_type: 10 unique values
city: 4687 unique values
city_fuel_economy: 100 unique values
daysonmarket: 1754 unique values
dealer_zip: 8328 unique values
engine_cylinders: 40 unique values
engine_displacement: 67 unique values
fleet: 2 unique values
frame_damaged: 2 unique values
franchise_dealer: 2 unique values
franchise_make: 49 unique values
front_legroom: 100 unique values
fuel_tank_volume: 181 unique values
fuel_type: 9 unique values
has_accidents: 2 unique values
height: 471 unique values
highway_fuel_economy: 99 unique values
horsepower: 455 unique values
isCab: 2 unique values
is_new: 2 unique values
length: 709 unique values
listed_date: 1749 unique values
listing_color: 15 unique values
major_options: 279973 unique values
make_name: 100 unique values
maximum_seating: 11 unique values
mileage: 197577 unique values
model_name: 1429 unique values
owner_count: 18 unique values
price: 87834 unique values
salvage: 2 unique values
seller_rating: 701 

In [24]:
# Data filtering and cleaning
print(f"Initial dataset shape: {cars_df.shape}")

# Drop records with missing mileage and/or price
cars_df = cars_df.dropna(subset=['mileage', 'price'])
print(f"After dropping missing mileage/price: {cars_df.shape}")

# Drop cars valued over $100k and under $1k
cars_df = cars_df[(cars_df['price'] >= 1000) & (cars_df['price'] <= 100000)]
print(f"After filtering price range ($1k-$100k): {cars_df.shape}")

# Drop mileage over 200k
cars_df = cars_df[cars_df['mileage'] <= 200000]
print(f"After filtering mileage (≤200k): {cars_df.shape}")

# Drop year values before 2001
cars_df = cars_df[cars_df['year'] >= 2001]
print(f"After filtering year (≥2001): {cars_df.shape}")

print(f"Final dataset shape: {cars_df.shape}")

Initial dataset shape: (3000000, 43)
After dropping missing mileage/price: (2855613, 43)
After filtering price range ($1k-$100k): (2839900, 43)
After filtering mileage (≤200k): (2820217, 43)
After filtering year (≥2001): (2808932, 43)
Final dataset shape: (2808932, 43)


## Feature Engineering

In [25]:
# Create date-based features from listed_date
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

# Extract date components
cars_df['listing_month'] = cars_df['listed_date'].dt.month
cars_df['listing_year'] = cars_df['listed_date'].dt.year
cars_df['listing_day_of_week'] = cars_df['listed_date'].dt.dayofweek

# Create derived features
cars_df['listing_season'] = cars_df['listing_month'].apply(get_season).astype('category')
cars_df['is_weekend_listing'] = (cars_df['listing_day_of_week'] >= 5).astype('boolean')

print("Created all date-based features: listing_month, listing_year, listing_day_of_week, listing_season, is_weekend_listing")

Created all date-based features: listing_month, listing_year, listing_day_of_week, listing_season, is_weekend_listing


In [26]:
# Create car age feature
cars_df['car_age'] = 2025 - cars_df['year']
print("Created car_age feature")

Created car_age feature


In [27]:
# Create mileage per year feature
cars_df['mileage_per_year'] = cars_df['mileage'] / cars_df['car_age']
cars_df['mileage_per_year'] = cars_df['mileage_per_year'].replace([np.inf, -np.inf], np.nan)
print("Created mileage_per_year feature")

Created mileage_per_year feature


In [28]:
# Create combined fuel efficiency score
cars_df['fuel_efficiency_combined'] = (cars_df['city_fuel_economy'] + cars_df['highway_fuel_economy']) / 2
print("Created fuel_efficiency_combined feature")

Created fuel_efficiency_combined feature


In [29]:
# This is approximate - you'd need actual weight data for precision
cars_df['power_per_displacement'] = cars_df['horsepower'] / cars_df['engine_displacement']
cars_df['power_per_displacement'] = cars_df['power_per_displacement'].replace([np.inf, -np.inf], np.nan)
print("Created power_per_displacement feature")

Created power_per_displacement feature


In [30]:
# Create luxury brand indicator
luxury_brands = ['BMW', 'Mercedes-Benz', 'Audi', 'Lexus', 'Acura', 'Infiniti', 
                'Cadillac', 'Lincoln', 'Porsche', 'Jaguar', 'Land Rover', 'Volvo']
cars_df['is_luxury_brand'] = cars_df['make_name'].isin(luxury_brands).astype('boolean')
print("Created is_luxury_brand feature")

Created is_luxury_brand feature


In [31]:
# Create market time categories
cars_df['market_time_category'] = pd.cut(cars_df['daysonmarket'], 
                                        bins=[0, 30, 90, 180, np.inf], 
                                        labels=['Quick_Sale', 'Normal', 'Slow', 'Very_Slow'])
print("Created market_time_category feature")

Created market_time_category feature


In [32]:
# Create high mileage indicator (>100k miles)
cars_df['high_mileage'] = (cars_df['mileage'] > 100000).astype('boolean')
print("Created high_mileage feature")

Created high_mileage feature


In [33]:
# One-hot encode low cardinality categorical columns
onehot_cols = ['body_type', 'fuel_type', 'transmission', 'wheel_system', 'listing_color']
cars_df = pd.get_dummies(cars_df, columns=onehot_cols, drop_first=True)
print(f"One-hot encoded columns: {onehot_cols}")
print(f"New shape after one-hot encoding: {cars_df.shape}")

One-hot encoded columns: ['body_type', 'fuel_type', 'transmission', 'wheel_system', 'listing_color']
New shape after one-hot encoding: (2808932, 90)


In [34]:
cars_df.drop(columns=['listed_date'], inplace=True)

In [35]:
# Convert all object columns to categorical
object_cols = cars_df.select_dtypes(include=['object']).columns.tolist()
print(f"Converting object columns to categorical: {object_cols}")

for col in object_cols:
    cars_df[col] = cars_df[col].astype('category')

print(f"Data types after conversion:")
print(cars_df.dtypes)

Converting object columns to categorical: ['city', 'dealer_zip', 'engine_cylinders', 'franchise_make', 'major_options', 'make_name', 'model_name', 'sp_name', 'torque', 'trim_name']
Data types after conversion:
back_legroom                         float16
city                                category
city_fuel_economy                    float16
daysonmarket                           int16
dealer_zip                          category
engine_cylinders                    category
engine_displacement                    Int16
fleet                                boolean
frame_damaged                        boolean
franchise_dealer                        bool
franchise_make                      category
front_legroom                        float16
fuel_tank_volume                     float16
has_accidents                        boolean
height                               float16
highway_fuel_economy                 float16
horsepower                             Int16
isCab                    

In [36]:
cars_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2808932 entries, 0 to 3000039
Data columns (total 89 columns):
 #   Column                            Dtype   
---  ------                            -----   
 0   back_legroom                      float16 
 1   city                              category
 2   city_fuel_economy                 float16 
 3   daysonmarket                      int16   
 4   dealer_zip                        category
 5   engine_cylinders                  category
 6   engine_displacement               Int16   
 7   fleet                             boolean 
 8   frame_damaged                     boolean 
 9   franchise_dealer                  bool    
 10  franchise_make                    category
 11  front_legroom                     float16 
 12  fuel_tank_volume                  float16 
 13  has_accidents                     boolean 
 14  height                            float16 
 15  highway_fuel_economy              float16 
 16  horsepower             

In [37]:
# Save dataframe to parquet file
cars_df.to_parquet('cars_df.parquet', compression='snappy', index=False)
print("Dataframe saved as cars_df.parquet")

Dataframe saved as cars_df.parquet


# Continue from here

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import ast
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from catboost import CatBoostRegressor, Pool
from sklearn.impute import SimpleImputer
from category_encoders import TargetEncoder
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load dataframe from parquet file
cars_df = pd.read_parquet('cars_df.parquet')
print(f"Dataframe loaded from parquet. Shape: {cars_df.shape}")

Dataframe loaded from parquet. Shape: (2808932, 89)


In [3]:
cars_df.drop(columns=['major_options'], inplace=True)

## Modeling

### Train-test split

In [4]:
X = cars_df.drop('price', axis=1) 
y = cars_df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Log transform target for better model performance
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Features: {X_train.shape[1]}")

Training set: 2247145 samples
Test set: 561787 samples
Features: 87


### LightGBM

In [ ]:
# LightGBM setup with GPU acceleration
lgb_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.1,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1,
    'random_state': 42
}

# Create LightGBM datasets
train_data = lgb.Dataset(X_train, label=y_train_log)
valid_data = lgb.Dataset(X_test, label=y_test_log, reference=train_data)

# Train model with early stopping
lgb_model = lgb.train(
    lgb_params,
    train_data,
    valid_sets=[train_data, valid_data],
    valid_names=['train', 'valid'],
    num_boost_round=5000,
    callbacks=[lgb.early_stopping(50), lgb.log_evaluation(100)]
)

# Make predictions
y_pred_log = lgb_model.predict(X_test)
y_pred = np.expm1(y_pred_log)  # Convert back from log scale

# Evaluate model
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"\nLightGBM Model Performance:")
print(f"MAE: ${mae:,.2f}")
print(f"RMSE: ${rmse:,.2f}")
print(f"R²: {r2:.4f}")
print(f"Mean Price: ${y_test.mean():,.2f}")

Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 0.100547	valid's rmse: 0.10236
[200]	train's rmse: 0.089236	valid's rmse: 0.092006
[300]	train's rmse: 0.0837621	valid's rmse: 0.0874044
[400]	train's rmse: 0.0806301	valid's rmse: 0.0850225
[500]	train's rmse: 0.0784668	valid's rmse: 0.0834647
[600]	train's rmse: 0.0769272	valid's rmse: 0.0825303
[700]	train's rmse: 0.0757523	valid's rmse: 0.0818422
[800]	train's rmse: 0.0748168	valid's rmse: 0.0813452
[900]	train's rmse: 0.0740438	valid's rmse: 0.0809799
[1000]	train's rmse: 0.0732945	valid's rmse: 0.0806238
[1100]	train's rmse: 0.0727287	valid's rmse: 0.0803664
[1200]	train's rmse: 0.0720592	valid's rmse: 0.0800291
[1300]	train's rmse: 0.0715563	valid's rmse: 0.0798066
[1400]	train's rmse: 0.0711686	valid's rmse: 0.0796862
[1500]	train's rmse: 0.0707137	valid's rmse: 0.0795093
[1600]	train's rmse: 0.0703594	valid's rmse: 0.0793996
[1700]	train's rmse: 0.0699729	valid's rmse: 0.079259
[1800]	train's rmse

### CatBoostRegressor

In [5]:
# CatBoost setup - handles categorical features automatically
cat_features = ['city', 'dealer_zip', 'engine_cylinders', 'franchise_make', 
                'make_name', 'model_name', 'sp_name', 'torque', 'transmission_display', 
                'trim_name', 'listing_season', 'market_time_category']

# Handle missing values comprehensively
# First, handle all columns with any type of missing values
for col in X_train.columns:
    if X_train[col].dtype.name == 'category':
        # For categorical columns
        if 'Unknown' not in X_train[col].cat.categories:
            X_train[col] = X_train[col].cat.add_categories(['Unknown'])
        if 'Unknown' not in X_test[col].cat.categories:
            X_test[col] = X_test[col].cat.add_categories(['Unknown'])
        X_train[col] = X_train[col].fillna('Unknown')
        X_test[col] = X_test[col].fillna('Unknown')
    elif X_train[col].dtype in ['float16', 'float32', 'float64', 'Float64', 'int16', 'int32', 'int64', 'Int16', 'Int32', 'Int64', 'Int8']:
        # For numeric columns
        fill_value = X_train[col].median()
        X_train[col] = X_train[col].fillna(fill_value)
        X_test[col] = X_test[col].fillna(fill_value)
    elif X_train[col].dtype == 'boolean':
        # For boolean columns
        X_train[col] = X_train[col].fillna(False)
        X_test[col] = X_test[col].fillna(False)

# Create CatBoost pools
train_pool = Pool(
    data=X_train,
    label=y_train_log,
    cat_features=cat_features
)

test_pool = Pool(
    data=X_test,
    label=y_test_log,
    cat_features=cat_features
)

In [6]:
# Initialize CatBoost model
catboost_model = CatBoostRegressor(
    iterations=10000,
    learning_rate=0.05,
    depth=16,
    loss_function='RMSE',
    eval_metric='RMSE',
    random_seed=42,
    task_type='GPU',
    devices='0',
    early_stopping_rounds=50,
    verbose=100
)

# Train model
catboost_model.fit(
    train_pool,
    eval_set=test_pool,
    plot=False
)

# Make predictions
y_pred_log_cat = catboost_model.predict(X_test)
y_pred_cat = np.expm1(y_pred_log_cat)

# Evaluate model
mae_cat = mean_absolute_error(y_test, y_pred_cat)
rmse_cat = np.sqrt(mean_squared_error(y_test, y_pred_cat))
r2_cat = r2_score(y_test, y_pred_cat)

print(f"\nCatBoost Model Performance:")
print(f"MAE: ${mae_cat:,.2f}")
print(f"RMSE: ${rmse_cat:,.2f}")
print(f"R²: {r2_cat:.4f}")

0:	learn: 0.5466615	test: 0.5455729	best: 0.5455729 (0)	total: 1.09s	remaining: 3h 1m 37s
100:	learn: 0.1094304	test: 0.1110177	best: 0.1110177 (100)	total: 1m 20s	remaining: 2h 11m 21s
200:	learn: 0.0954671	test: 0.0985244	best: 0.0985244 (200)	total: 2m 34s	remaining: 2h 5m 37s
300:	learn: 0.0877763	test: 0.0924021	best: 0.0924021 (300)	total: 3m 49s	remaining: 2h 3m 15s
400:	learn: 0.0825198	test: 0.0889158	best: 0.0889158 (400)	total: 5m 2s	remaining: 2h 33s
500:	learn: 0.0781414	test: 0.0862688	best: 0.0862688 (500)	total: 6m 14s	remaining: 1h 58m 21s
600:	learn: 0.0746335	test: 0.0844361	best: 0.0844361 (600)	total: 7m 27s	remaining: 1h 56m 32s
700:	learn: 0.0714305	test: 0.0828789	best: 0.0828789 (700)	total: 8m 40s	remaining: 1h 55m 2s
800:	learn: 0.0684480	test: 0.0816048	best: 0.0816048 (800)	total: 9m 53s	remaining: 1h 53m 32s
900:	learn: 0.0658497	test: 0.0805990	best: 0.0805990 (900)	total: 11m 6s	remaining: 1h 52m 13s
1000:	learn: 0.0635484	test: 0.0797366	best: 0.0797366

In [7]:
catboost_model.save_model('catboost_model.cbm')
print("CatBoost model saved as 'catboost_model.cbm'")

CatBoost model saved as 'catboost_model.cbm'


In [5]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

# CatBoost hyperparameter tuning with GPU
catboost_params = {
    'iterations': randint(1000, 5000),
    'learning_rate': uniform(0.01, 0.2),
    'depth': randint(4, 10),
    'l2_leaf_reg': uniform(1, 10),
    'border_count': randint(32, 255),
    'bagging_temperature': uniform(0, 1),
    'random_strength': uniform(0, 10)
}

# Base CatBoost model with GPU
catboost_base = CatBoostRegressor(
    loss_function='RMSE',
    eval_metric='RMSE',
    random_seed=42,
    task_type='GPU',
    devices='0',
    early_stopping_rounds=50,
    verbose=False
)

import time
from datetime import datetime

# Add progress tracking
class ProgressCallback:
    def __init__(self):
        self.start_time = time.time()
        self.iteration = 0
    
    def __call__(self, study, trial):
        self.iteration += 1
        elapsed = time.time() - self.start_time
        print(f"[{datetime.now().strftime('%H:%M:%S')}] Completed {self.iteration}/20 iterations in {elapsed/60:.1f}min")

# If you need to restart with progress tracking:
catboost_random = RandomizedSearchCV(
    catboost_base,
    catboost_params,
    n_iter=20,
    cv=3,
    scoring='neg_mean_squared_error',
    random_state=42,
    n_jobs=1,
    verbose=2  # This shows sklearn progress
)

print("Starting CatBoost hyperparameter tuning with GPU...")
catboost_random.fit(
    X_train, y_train_log,
    cat_features=cat_features,
    eval_set=[(X_test, y_test_log)]
)

print(f"Best CatBoost parameters: {catboost_random.best_params_}")

# Train final model with best parameters
catboost_optimized = catboost_random.best_estimator_

# Make predictions
y_pred_log_cat_tuned = catboost_optimized.predict(X_test)
y_pred_cat_tuned = np.expm1(y_pred_log_cat_tuned)

# Evaluate tuned model
mae_cat_tuned = mean_absolute_error(y_test, y_pred_cat_tuned)
rmse_cat_tuned = np.sqrt(mean_squared_error(y_test, y_pred_cat_tuned))
r2_cat_tuned = r2_score(y_test, y_pred_cat_tuned)

print(f"\nTuned CatBoost Model Performance:")
print(f"MAE: ${mae_cat_tuned:,.2f}")
print(f"RMSE: ${rmse_cat_tuned:,.2f}")
print(f"R²: {r2_cat_tuned:.4f}")

Starting CatBoost hyperparameter tuning with GPU...
Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] END bagging_temperature=0.3745401188473625, border_count=124, depth=6, iterations=2095, l2_leaf_reg=6.986584841970366, learning_rate=0.0412037280884873, random_strength=1.5599452033620265; total time= 1.9min
[CV] END bagging_temperature=0.3745401188473625, border_count=124, depth=6, iterations=2095, l2_leaf_reg=6.986584841970366, learning_rate=0.0412037280884873, random_strength=1.5599452033620265; total time= 2.1min
[CV] END bagging_temperature=0.3745401188473625, border_count=124, depth=6, iterations=2095, l2_leaf_reg=6.986584841970366, learning_rate=0.0412037280884873, random_strength=1.5599452033620265; total time= 2.0min
[CV] END bagging_temperature=0.05808361216819946, border_count=119, depth=8, iterations=4171, l2_leaf_reg=2.428668179219408, learning_rate=0.14017769458977059, random_strength=0.5641157902710026; total time= 4.9min
[CV] END bagging_temperature=0.05

Best CatBoost parameters: {'bagging_temperature': np.float64(0.4667628932479799), 'border_count': 82, 'depth': 8, 'iterations': 3888, 'l2_leaf_reg': np.float64(1.6505159298527952), 'learning_rate': np.float64(0.19977710745066668), 'random_strength': np.float64(9.656320330745594)}

### Error-Modeling